In [ ]:
import os
import json
from typing import Final
from yfpy.query import YahooFantasySportsQuery
from dotenv import load_dotenv
import pandas as pd
import uuid

auth_dir = "/Users/petepritchard/Documents/Projects/ff-league-app/bot/auth"

# Load environment variables from .env file
load_dotenv()

# Fetch Yahoo API credentials from environment variables
CONSUMER_KEY: Final[str] = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET: Final[str] = os.getenv('CONSUMER_SECRET')
REFRESH_TOKEN: Final[str] = os.getenv('REFRESH_TOKEN')

# Fetch league information
query = YahooFantasySportsQuery(
    auth_dir, 
    league_id='49754', # Fill in league_id
    game_code='nfl',   # Leave this as is
    game_id='423',     # Fill in current game id
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    )

In [27]:
transactions = query.get_league_transactions()
recent_transaction = transactions[1].to_json()
data = json.loads(recent_transaction)

# Retrieving the values
player_id = data['players']['player']['player_id']
destination_team_key = data['players']['player']['transaction_data']['destination_team_key']
transaction_type = data['players']['player']['transaction_data']['type']

# Printing the retrieved values
print("Player ID:", player_id)
print("Destination Team Key:", destination_team_key)
print("Transaction Type:", transaction_type)

## Pulling data from Yahoo for Matchups Table

In [71]:
# Initialize an empty list to store records
records = []

for i in range(1, 3):
    # Query data from yahoo
    matchups = query.get_league_matchups_by_week(i)
    # Loop through matchup data
    for i in range(len(matchups)):
        matchup = matchups[i].to_json() # matchup[i] is a yfpy object
        matchup_dict = json.loads(matchup)
        
        # Generate a unique matchup_id (UUID)
        matchup_id = str(uuid.uuid4())

        # Extract information for each team in the matchup
        for team in matchup_dict.get('teams', []):
            team_info = team.get('team', {})
            record = {
                'primary_key': str(uuid.uuid4()),  # Unique primary key for each row
                'matchup_id': matchup_id,
                'season_key': team_info.get('team_key')[:3], # First 3 characters in team key
                'week': matchup_dict.get('week'),
                'is_consolation': matchup_dict.get('is_consolation'),
                'is_playoffs': matchup_dict.get('is_playoffs'),
                'is_tied': matchup_dict.get('is_tied'),
                'winner_team_key': matchup_dict.get('winner_team_key'),
                'team_key': team_info.get('team_key'),
                'team_total_points': team_info.get('team_points', {}).get('total'),
                'team_projected_points': team_info.get('team_projected_points', {}).get('total')
            }
            
            # Add the record to the list
            records.append(record)

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())

                            primary_key                            matchup_id  \
0  423bb7d1-1c33-4b36-b457-156a7124b87e  6dbf59fa-4e35-4170-adab-1527a3f11f19   
1  78de06a1-d6be-466d-b0e0-ce97fae9677e  6dbf59fa-4e35-4170-adab-1527a3f11f19   
2  943c5aff-a0c8-4d92-aae1-ae69045ac7b6  aed76cc7-3954-4bd5-afcf-6ce4601b3827   
3  b0e3b5c7-8b64-480e-aaea-3d176c9f89bf  aed76cc7-3954-4bd5-afcf-6ce4601b3827   
4  f6f1900c-20c9-4615-b75f-5f5a421ef108  618208f4-129d-40ad-9ed0-c9f30c153b8a   

  season_key  week  is_consolation  is_playoffs  is_tied  winner_team_key  \
0        423     1               0            0        0  423.l.49754.t.7   
1        423     1               0            0        0  423.l.49754.t.7   
2        423     1               0            0        0  423.l.49754.t.2   
3        423     1               0            0        0  423.l.49754.t.2   
4        423     1               0            0        0  423.l.49754.t.3   

          team_key  team_total_points  team_projec

In [69]:
df.to_clipboard()

In [ ]:
matchups_table = """
CREATE TABLE Matchups (
    primary_key INT AUTO_INCREMENT PRIMARY KEY,
    matchup_id INT,
    season_key INT NOT NULL,
    week INT NOT NULL,
    is_consolation BOOLEAN,
    is_playoffs BOOLEAN,
    is_tied BOOLEAN,
    winner_team_key VARCHAR(50),
    team_key VARCHAR(50),
    team_total_points INT,
    team_projected_points INT,
    UNIQUE(season, week, winner_team_key) -- Ensuring unique combination of season, week, and winner team
);
"""

## Seasons Table

In [94]:
seasons_dict = {
    "season_key": ['257', '273', '314', '331', '348', '259', '317', '380', '380', '399', '406', '414', '423', '449'],
    "season": ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
}

df_seasons = pd.DataFrame(seasons_dict)
df_seasons.head()

,season_key,season
0,257,2011
1,273,2012
2,314,2013
3,331,2014
4,348,2015


In [ ]:
seasons_table = """
CREATE TABLES Seasons (
    season_key INT PRIMARY KEY,
    season INT
    UNIQUE(season_key, season) -- Ensuring unique combination of season and season_key
);
"""

## Drafts Table

In [99]:
draft = query.get_league_draft_results()
print(type(draft[1]))

<class 'yfpy.models.DraftResult'>


In [100]:
print(draft[1])

DraftResult({
  "pick": 2,
  "round": 1,
  "team_key": "423.l.49754.t.6",
  "player_key": "423.p.33393"
})


In [104]:
records = []

for i in range(len(draft)):
    pick = draft[i].to_json()
    pick_dict = json.loads(pick)
    record = {
        'draft_id': str(uuid.uuid4()),
        'pick': pick_dict.get('pick'),
        'round': pick_dict.get('round'),
        'team_key': pick_dict.get('team_key'),
        'player_key': pick_dict.get('player_key')
    }

    records.append(record)

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())    

                               draft_id  pick  round          team_key  \
0  14c08d76-5222-4b87-9791-3bcf4247ae75     1      1  423.l.49754.t.12   
1  aa9b6347-48b4-4a8f-950f-3522b81908ab     2      1   423.l.49754.t.6   
2  281d24bc-bb6c-43f0-896b-5df49067222b     3      1   423.l.49754.t.4   
3  0f0ab24c-5ccf-438b-85bb-9f28638adea7     4      1  423.l.49754.t.10   
4  6ea91085-0eb2-44cc-a9cc-b5dc1d88a9dc     5      1   423.l.49754.t.2   

    player_key  
0  423.p.30121  
1  423.p.33393  
2  423.p.32692  
3  423.p.31005  
4  423.p.30423  


## Players Table

In [107]:
players = query.get_league_players()
print(players)

2024-08-13 20:52:07.720 - ERROR - query.py - yfpy.query:290 - No data found when attempting extraction from fields: ['league', 'players']


[Player({
  "bye_weeks": {
    "week": 5
  },
  "display_position": "QB",
  "editorial_player_key": "nfl.p.5228",
  "editorial_team_abbr": "TB",
  "editorial_team_full_name": "Tampa Bay Buccaneers",
  "editorial_team_key": "nfl.t.27",
  "editorial_team_url": "https://sports.yahoo.com/nfl/teams/tampa-bay/",
  "eligible_positions": [
    "QB"
  ],
  "headshot": {
    "size": "small",
    "url": "https://s.yimg.com/iu/api/res/1.2/Ol_o6nNm9irLqsJ.2ZxQHg--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/09082022/5228.png"
  },
  "image_url": "https://s.yimg.com/iu/api/res/1.2/Ol_o6nNm9irLqsJ.2ZxQHg--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/09082022/5228.png",
  "is_keeper": {
    "status": false,
    "cost": false,
    "kept": false
  },
  "is_undroppable": 0,
  "name": {
    "a

In [110]:
print(players[1])

Player({
  "bye_weeks": {
    "week": 7
  },
  "display_position": "QB",
  "editorial_player_key": "nfl.p.7200",
  "editorial_team_abbr": "NYJ",
  "editorial_team_full_name": "New York Jets",
  "editorial_team_key": "nfl.t.20",
  "editorial_team_url": "https://sports.yahoo.com/nfl/teams/ny-jets/",
  "eligible_positions": [
    "QB"
  ],
  "headshot": {
    "size": "small",
    "url": "https://s.yimg.com/iu/api/res/1.2/wBl5RxV59_1mMQkQHhlz_A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08022023/7200.png"
  },
  "image_url": "https://s.yimg.com/iu/api/res/1.2/wBl5RxV59_1mMQkQHhlz_A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08022023/7200.png",
  "injury_note": "Achilles",
  "is_keeper": {
    "status": false,
    "cost": false,
    "kept": false
  },
  "is_undroppable": 0

In [113]:
records = []

for i in range(len(players)):
    player = players[i].to_json()
    player_dict = json.loads(player)
    record = {
        'player_id': str(uuid.uuid4()),
        'player_key': player_dict.get('player_key'),
        'full_name': player_dict.get('name', {}).get('full'),
        'bye_week': player_dict.get('bye_weeks', {}).get('week'),
        'position': player_dict.get('display_position'),
        'team_abbr': player_dict.get('editorial_team_abbr'),
        'status': player_dict.get('status')
    }

    records.append(record)
    
# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())   

   player_key       full_name  bye_week position team_abbr status
0  423.p.5228       Tom Brady         5       QB        TB     NA
1  423.p.7200   Aaron Rodgers         7       QB       NYJ      Q
2  423.p.7520    Robbie Gould         9        K        SF     NA
3  423.p.7777  Marcedes Lewis        13       TE       Chi   None
4  423.p.8432       Nick Folk         7        K       Ten   None


In [114]:
df.to_clipboard()

## Managers Table